In [1]:
import random
import time

import pandas as pd
import numpy as np

from sklearn import preprocessing
from collections import deque

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping


import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [2]:
SEQ_LEN = 180
FUTURE_PERIOD_PREDICT = 3

RATIO_TO_PREDICT = "ETH-USD"

LR=0.001 # 0.001
DECAY=1e-5 # 1e-6

EPOCHS = 100
BATCH_SIZE = 64
LSTM_LAYERS = [4, 2]
LSTM_LAYER_SIZES = [128, 64]
DENSE_LAYER_SIZES = [32, 16]

# NAME = f"{RATIO_TO_PREDICT}-{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{BATCH_SIZE}-BATCH-{LSTM_LAYERS}{int(time.time())}"

In [3]:
def classify(current, future):
    # This if statement essentially says that if the future price is more than the current one, its a good thing.
    if float(future) > float(current):
        return 1
    else:
        return 0


def preprocess_df(df):
    df = df.drop(columns="future")
    
    for col in df.columns:
        if col != "target":
            df[col] = df[col].pct_change() # get the percentage change.
            df.dropna(inplace=True)
            df[col] = preprocessing.scale(df[col].values) # Normalises the data, between 0 and 1. 
            
    df.dropna(inplace=True)
    
    sequential_data = []
    prev_days = deque(maxlen=SEQ_LEN) # once it has the SEQ_LEN number of values it will return a list
    
    for i in df.values:
        prev_days.append([n for n in i[:-1]])    
        if len(prev_days) == SEQ_LEN:
            sequential_data.append([np.array(prev_days), i[-1]])
            
    random.shuffle(sequential_data)
    
    buys = []
    sells = []
    
    for seq, target in sequential_data:
        if target == 0:
            sells.append([seq, target])
        elif target == 1:
            buys.append([seq, target])
            
    random.shuffle(buys)
    random.shuffle(sells)
    
    lower = min(len(buys), len(sells))
    buys = buys[:lower]
    sells = sells[:lower]
    
    sequential_data = buys+sells
    random.shuffle(sequential_data)
    
    X = []
    y = []
    
    for seq, target in sequential_data:
        X.append(seq)
        y.append(target)
    return np.array(X), y

In [4]:
main_df = pd.DataFrame()

ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]

for ratio in ratios:
    dataset = f"crypto_data/{ratio}.csv"
    
    df = pd.read_csv(dataset, names=["time", "low", "high", "open", "close", "volume"])
    
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)
    
    df.set_index("time", inplace=True)
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    

    if len(main_df) == 0:
        main_df = df
    else:
        main_df = main_df.join(df)
    
main_df["future"] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT)
main_df['target'] = list(map(classify, main_df[f"{RATIO_TO_PREDICT}_close"], main_df["future"]))

print(main_df[[f"{RATIO_TO_PREDICT}_close", "future", "target"]].head(5))


times =sorted(main_df.index.values)
last_5pct = times[-int(0.05*len(times))]


            ETH-USD_close     future  target
time                                        
1528968660            NaN  485.75000       0
1528968720      486.01001  486.00000       0
1528968780      486.00000  486.00000       0
1528968840      485.75000  485.98999       1
1528968900      486.00000  485.98999       0


In [5]:
# Getting validation data. The last 5% of data in the dataframe.
times =sorted(main_df.index.values)
last_5pct = times[-int(0.05*len(times))]

validation_main_df = main_df[(main_df.index >= last_5pct)]
main_df = main_df[(main_df.index < last_5pct)]

train_x, train_y = preprocess_df(main_df)
validation_x, validation_y = preprocess_df(validation_main_df)

print(f"train data: {len(train_x)} validation: {len(validation_x)}")
print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")
print(f"VALIDATION Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")

train data: 74084 validation: 3144
Dont buys: 37042, buys: 37042
VALIDATION Dont buys: 1572, buys: 1572


In [6]:

for LSTM_LAYER in LSTM_LAYERS:
    for LSTM_LAYER_SIZE in LSTM_LAYER_SIZES:
        for DENSE_LAYER_SIZE in DENSE_LAYER_SIZES: 
            
            NAME = f"{RATIO_TO_PREDICT}-{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{BATCH_SIZE}-BATCH-{LSTM_LAYER}-LSTM-{DENSE_LAYER_SIZE}-DENSE_NODE-{LSTM_LAYER_SIZE}-LSTM_NODE-{LR}-LEARN_RATE-{DECAY}-DECAY-{int(time.time())}"
            
            tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
            
            early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=1, mode='auto')
            
            model = Sequential()
            
            for l in range(LSTM_LAYER):
                if l < LSTM_LAYER -1:
                    model.add(LSTM(LSTM_LAYER_SIZE, input_shape=(train_x.shape[1:]), return_sequences=True))
                    model.add(Dropout(0.2))
                    model.add(BatchNormalization())
                else:
                    model.add(LSTM(LSTM_LAYER_SIZE, input_shape=(train_x.shape[1:])))
                    model.add(Dropout(0.3))
                    model.add(BatchNormalization())
            
            model.add(Dense(DENSE_LAYER_SIZE, activation="relu"))
            model.add(Dropout(0.3))
            
            model.add(Dense(2, activation="softmax"))
            
            opt = tf.keras.optimizers.Adam(learning_rate=LR, decay=DECAY)

            model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy']) # consider binary crossentropy for loss
            
            filepath = "RNN_Final-{epoch:02d}-{val_accuracy:.3f}"
            checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor="val_accuracy", verbose=1, save_best_only=True, mode="max"))
            
            train_x = np.asarray(train_x)
            train_y = np.asarray(train_y)
            validation_x = np.asarray(validation_x)
            validation_y = np.asarray(validation_y)

            history = model.fit(train_x, train_y, 
                                batch_size=BATCH_SIZE, 
                                epochs=EPOCHS, 
                                validation_data=(validation_x, validation_y), 
                                callbacks=[tensorboard, checkpoint, early_stop])

            # model.save(f'{NAME}.model', save_format="h5")
"""model = Sequential()
model.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2)) # if anything weird, try putting this back to 0.2
model.add(BatchNormalization())

model.add(LSTM(128, input_shape=(train_x.shape[1:])))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(2, activation="softmax"))

opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)

model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy']) # consider binary crossentropy for loss

tensorboard = TensorBoard(log_dir=f'logs/{NAME}')

filepath = "RNN_Final-{epoch:02d}-{val_accuracy:.3f}"
checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor="val_accuracy", verbose=1, save_best_only=True, mode="max"))

train_x = np.asarray(train_x)
train_y = np.asarray(train_y)
validation_x = np.asarray(validation_x)
validation_y = np.asarray(validation_y)

history = model.fit(train_x, train_y, 
                    batch_size=BATCH_SIZE, 
                    epochs=EPOCHS, 
                    validation_data=(validation_x, validation_y), 
                    callbacks=[tensorboard, checkpoint])"""

Epoch 1/100
1157/1158 [============================>.] - ETA: 0s - loss: 0.7304 - accuracy: 0.5044INFO:tensorflow:Assets written to: models\RNN_Final-01-0.501.model\assets


INFO:tensorflow:Assets written to: models\RNN_Final-01-0.501.model\assets


1158/1158 [==============================] - 119s 91ms/step - loss: 0.7304 - accuracy: 0.5044 - val_loss: 0.6944 - val_accuracy: 0.5010
Epoch 2/100
1158/1158 [==============================] - ETA: 0s - loss: 0.6929 - accuracy: 0.5195INFO:tensorflow:Assets written to: models\RNN_Final-02-0.501.model\assets


INFO:tensorflow:Assets written to: models\RNN_Final-02-0.501.model\assets


1158/1158 [==============================] - 103s 89ms/step - loss: 0.6929 - accuracy: 0.5195 - val_loss: 0.6948 - val_accuracy: 0.5010
Epoch 3/100
1157/1158 [============================>.] - ETA: 0s - loss: 0.6913 - accuracy: 0.5277INFO:tensorflow:Assets written to: models\RNN_Final-03-0.511.model\assets


...

INFO:tensorflow:Assets written to: models\RNN_Final-07-0.563.model\assets


1158/1158 [==============================] - 39s 33ms/step - loss: 0.6793 - accuracy: 0.5647 - val_loss: 0.6848 - val_accuracy: 0.5633
Epoch 8/100
1156/1158 [============================>.] - ETA: 0s - loss: 0.6776 - accuracy: 0.5716INFO:tensorflow:Assets written to: models\RNN_Final-08-0.561.model\assets


INFO:tensorflow:Assets written to: models\RNN_Final-08-0.561.model\assets


1158/1158 [==============================] - 39s 34ms/step - loss: 0.6776 - accuracy: 0.5715 - val_loss: 0.6808 - val_accuracy: 0.5611
Epoch 8: early stopping


'model = Sequential()\nmodel.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))\nmodel.add(Dropout(0.2))\nmodel.add(BatchNormalization())\n\nmodel.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))\nmodel.add(Dropout(0.2)) # if anything weird, try putting this back to 0.2\nmodel.add(BatchNormalization())\n\nmodel.add(LSTM(128, input_shape=(train_x.shape[1:])))\nmodel.add(Dropout(0.2))\nmodel.add(BatchNormalization())\n\nmodel.add(Dense(32, activation="relu"))\nmodel.add(Dropout(0.2))\n\nmodel.add(Dense(2, activation="softmax"))\n\nopt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)\n\nmodel.compile(loss=\'sparse_categorical_crossentropy\', optimizer=opt, metrics=[\'accuracy\']) # consider binary crossentropy for loss\n\ntensorboard = TensorBoard(log_dir=f\'logs/{NAME}\')\n\nfilepath = "RNN_Final-{epoch:02d}-{val_accuracy:.3f}"\ncheckpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor="val_accuracy", verbose=1, save_best